In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
import sys
import time
from IPython.display import display

import verdict

verdict.set_loglevel('error')
v = verdict.presto('presto')


def new_figure():
    return go.FigureWidget(data=go.Bar())

def updateFigure(fig, df):
    cols = df.columns
    if len(cols) == 1:
        fig.data[0].x = ['value']
        fig.data[0].y = df[cols[0]]
    else:
        fig.data[0].x = df.index
        fig.data[0].y = df[cols[-1]]

def sql(query):
    fig = new_figure()
    display(fig)
    start = time.time()
    result = v.sql(query)
    if len(result.columns) == 1:
        display(result)
    else:
        updateFigure(fig, result)
    print(f"elapsed time: {time.time() - start} seconds.")
    
def presto_sql(query):
    sql("bypass " + query)

def sql_stream(query):
    fig = new_figure()
    display(fig)
    result_itr = v.sql_stream(query)
    for i, result in enumerate(result_itr):
        if i < 5:
            if len(result.columns) == 1:
                display(result)
            else:
                updateFigure(fig, result)
                time.sleep(0.1)
        else:
            break


In [5]:
sql("""\
BYPASS SELECT ship_year, count(*)
FROM (
    SELECT year(l_shipdate) AS ship_year
    FROM hive.tpch_sf100.lineitem
)
GROUP BY ship_year
ORDER BY ship_year
""")

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'e307480b-6eb7-42c4-b739-135a296a5981'}], 'layout': {'templ…

elapsed time: 12.586604118347168 seconds.


In [4]:
sql("""\
BYPASS SELECT ship_year, ship_month, count(*)
FROM (
    SELECT year(l_shipdate) AS ship_year, month(l_shipdate) AS ship_month
    FROM hive.tpch_sf100.lineitem
)
GROUP BY ship_year, ship_month
ORDER BY ship_year, ship_month
""")

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'e0e611f9-bd08-4e22-a664-918f6f379aa1'}], 'layout': {'templ…

elapsed time: 44.26411175727844 seconds.
